In [1]:
import numpy as np
from numpy import array
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

Using TensorFlow backend.


In [5]:
length = 5 
arr = array([i/10 for i in range(0, 10,2)])

In [6]:
print(arr)

[0.  0.2 0.4 0.6 0.8]


### One to one model

In [32]:
x = arr.reshape(length, 1, 1) # input is 3D for LSTM always
y = arr.reshape(length, 1) #the output shape of model 

In [33]:
print(x.shape)

(5, 1, 1)


In [34]:
print(y.shape)

(5, 1)


In [35]:
units = 5 
dense_units= 1  #works fine
#dense_units= 2 # will try to give output as (batch_size,2) but defined is (5,1) in y.shape

model = Sequential()
model.add(LSTM(units, input_shape=(1,1), return_sequences= False)) #output is (batch_size,units)
model.add(Dense(dense_units))#gets input (None, units). Then (None, units) * (units, dense_units) => output (None, dense_units)
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(x, y, epochs = 4, batch_size = 5, verbose = 2)
print(model.summary())

ValueError: Error when checking target: expected dense_9 to have shape (2,) but got array with shape (1,)

In [19]:
X = [0.1, 0.3, 0.5, .7, .9 , 0.9, 0.99]  # only 5 values are used for printing of output ??!!! 
result = model.predict(x, batch_size=1, verbose=0)
for value in result:
	print('%.1f' % value)

0.0
0.0
0.0
0.1
0.1


### Many to one model without TimeDistributed wrapper
output one vector rather build out an output sequence one step at a time. model has one input with 5 time steps.

In [40]:
x = arr.reshape(1,5,1) # 1 input with 5 timesteps
y = arr.reshape(1,5) # output shape = (1,5) : because 1 vector with 5 timesteps is the output

In [41]:
print(x.shape)
print(y.shape)

(1, 5, 1)
(1, 5)


In [42]:
units= 5
dense_units = 5
model = Sequential()
model.add(LSTM(units, input_shape= (5, 1), return_sequences = False)) # output is (none, units)
model.add(Dense(dense_units))  # (none,units)* (units, dense_units) => output is  (none, dense_units)
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(x, y, epochs = 6, batch_size = 1, verbose = 2)
print(model.summary())

Epoch 1/6
 - 2s - loss: 0.2084
Epoch 2/6
 - 0s - loss: 0.2059
Epoch 3/6
 - 0s - loss: 0.2038
Epoch 4/6
 - 0s - loss: 0.2016
Epoch 5/6
 - 0s - loss: 0.1993
Epoch 6/6
 - 0s - loss: 0.1970
Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (None, 5)                 140       
_________________________________________________________________
dense_7 (Dense)              (None, 5)                 30        
Total params: 170
Trainable params: 170
Non-trainable params: 0
_________________________________________________________________
None


In [43]:
units= 5
dense_units = 5
model = Sequential()
model.add(LSTM(units, input_shape= (5, 1), return_sequences = True)) # output is (none, timesteps, units)
model.add(Dense(dense_units))  # (none, timesteps ,units)* (units, dense_units) => output is  (none, timesteps, dense_units)
model.compile(loss='mean_squared_error', optimizer='adam') 
model.fit(x, y, epochs = 6, batch_size = 1, verbose = 2)  #but y is shaped as (1,5) hence error
print(model.summary())

ValueError: Error when checking target: expected dense_8 to have 3 dimensions, but got array with shape (1, 5)

### Many to many Sequence prediction with TimeDistributed Sequence

In [7]:
timesteps = 4 

In [9]:
arr2 = array([i/10 for i in range(0, 20,1)])

In [10]:
print(arr2)

[0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1.  1.1 1.2 1.3 1.4 1.5 1.6 1.7
 1.8 1.9]


In [36]:
x = arr2.reshape(5,timesteps, 1)
y = arr2.reshape(5,timesteps, 1)

In [27]:
timesteps = 4 
dense_units =1 
units = 2

In [28]:
model = Sequential()
model.add(LSTM(units, input_shape=(timesteps, 1), return_sequences = True)) # output is (none, timesteps, units)
model.add(TimeDistributed(Dense(dense_units))) # (none, timesteps, units) * (units, timesteps ,dense_units) = (none, timesteps, dense_units)
# for each time step dense_units no of timsesteps are output
# The TimeDistributed achieves this trick by applying the same Dense layer (same weights) to the LSTMs outputs for one time step at a time.
model.compile(loss='mean_squared_error', optimizer='adam')
print(model.summary())

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 4, 2)              32        
_________________________________________________________________
time_distributed_6 (TimeDist (None, 4, 1)              3         
Total params: 35
Trainable params: 35
Non-trainable params: 0
_________________________________________________________________
None


In [39]:
model.fit(x, y, epochs=5, batch_size=1, verbose=2)
# evaluate
result = model.predict(x, batch_size=1, verbose=0)
for value in result[0,:,0]:
	print('%.1f' % value)

Epoch 1/5
 - 0s - loss: 1.6502
Epoch 2/5
 - 0s - loss: 1.6222
Epoch 3/5
 - 0s - loss: 1.6056
Epoch 4/5
 - 0s - loss: 1.5832
Epoch 5/5
 - 0s - loss: 1.5619
0.1
0.1
0.0
0.0
